In [ ]:
%load_ext autoreload
%autoreload 2
from utils.chess_gameplay import Agent, play_game, play_tournament
import torch
import yaml
import sys

In [ ]:
# Initialize a pair of agents. These agents are initialized with no model to make judgements
# about the move options presented to them, so will randomly select from their set of legal moves.

USE_ARTEM_MODEl = False
if USE_ARTEM_MODEl:
    #model_config = yaml.safe_load(open("/root/outputs/chess_interrupt/a2ce1d50-bbdf-4d46-9944-9a651feeba0a/model_config.yaml"))
    model_config = yaml.safe_load(open("/root/outputs/chess_conv_7/a2ce1d50-bbdf-4d46-9944-9a651feeba0a/model_config.yaml"))
else:
    model_config = yaml.safe_load(open("models/transformer.yaml"))

if USE_ARTEM_MODEl:
    sys.path.append("/root/artem/chess-hackathon/models")
    #from rel2d_transformer import Model
    from convolutional import Model
else:
    #from models.convolutional import Model
    from models.rel2d_transformer import Model
model = Model(**model_config)

if USE_ARTEM_MODEl:
    checkpoint = torch.load("/root/outputs/chess_conv_7/a2ce1d50-bbdf-4d46-9944-9a651feeba0a/checkpoint.pt")
else:
    checkpoint = torch.load("out_reltrans/checkpoint.pt")
model.load_state_dict(checkpoint["model"])

agents = [Agent(model, p=0.1), Agent(model, p=0.5, k=10)]

In [ ]:
# Now we can make our random agents play a game against each other. They're not very good and they don'try
# actually even know what checkmate is, so we're pretty unlikely to get anything other than a draw.

game_result = play_game(
    table = 1,                                              # Used to send games to one tournament table or another
    agents = {'white': agents[0], 'black': agents[1]},      # We specify which agent is playing with which pieces
    max_moves = 50,                                          # Play until each agent has made up to 10 moves
    min_seconds_per_move = 5,                             # Artificially slow the game down for better watching
    verbose = True,                                         # Tell us what moves each agent makes and the outcome
    poseval = True                                          # Use stockfish to evaluate each position and fancy plot
)

white_points, black_points = game_result["white"]["points"], game_result["black"]["points"]
print(f"White: {white_points}, Black: {black_points}")

In [ ]:
# We can also play a "tournament" which is a series of games in which a pair of agents take turns in playing
# as black and then as white. Ensuring each agent gets a turn at white before deciding the winner removes the
# inherent bias / advantage of playing as white. On gameday we use stockfish to evaluate draw positions and 
# decide which player has the better position. If agents draw at the end of each two games then another two
# games are played and so on until eventually one agent wins decisively.

tournament_results, tournament_game_results = play_tournament(
    table = 1,
    agents = agents,
    max_games = 4,
    max_moves = 2,
    min_seconds_per_move = 0.1,
    verbose=False,
    poseval = True
)

agent0_points, agent1_points = tournament_results["agent0"], tournament_results["agent1"]
print(f"Agent0: {agent0_points}, Black: {agent1_points}")

In [ ]:
# Now let's check back in with our training model! First load in the trained model weights

import torch
import yaml
from model import Model

model_config = yaml.safe_load(open("model_config.yaml"))
model = Model(**model_config)

checkpoint = torch.load("/root/outputs/chess-eval/<experiment_id>/checkpoint.pt", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint["model"])

In [ ]:
# Now we can inject our model into one of our agents, and play a game! (hopefully our model-powered agent wins!)

agents = [Agent(model), Agent()]

game_result = play_game(
    table = 1,                                              # Used to send games to one tournament table or another
    agents = {'white': agents[0], 'black': agents[1]},      # We specify which agent is playing with which pieces
    max_moves = 5,                                          # Play until each agent has made up to 10 moves
    min_seconds_per_move = 0.1,                             # Artificially slow the game down for better watching
    verbose = True,                                         # Tell us what moves each agent makes and the outcome
    poseval = True                                          # Use stockfish to evaluate each position and fancy plot
)

In [ ]:
# And also play them in a fair tournament

tournament_results, tournament_game_results = play_tournament(
    table = 1,
    agents = agents,
    max_games = 4,
    max_moves = 2,
    min_seconds_per_move = 0.1,
    verbose=False,
    poseval = True
)

agent0_points, agent1_points = tournament_results["agent0"], tournament_results["agent1"]
print(f"Agent0: {agent0_points}, Black: {agent1_points}")